In [7]:
import random
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [8]:
class Team:
    def __init__(self, founder):
        self.founder = founder
        self.members = {founder}
        self.a = random.uniform(0, 0.5)
        self.b = random.uniform(0.75, 1.25)
        self.beta = random.uniform(1.5, 2)

    def output(self, effort, total_effort_prev):
        total_effort = total_effort_prev + effort
        return self.a * total_effort + self.b * total_effort ** self.beta

    def add_member(self, agent):
        self.members.add(agent)

    def remove_member(self, agent):
        self.members.remove(agent)

In [9]:
class Agent:
    def __init__(self, theta, omega, network):
        self.theta = theta
        self.omega = omega
        self.network = network
        self.team = None
        self.effort = 0
        self.utility = 0

    def choose_effort(self, team_output, team_size, total_effort_prev):
        def utility(effort):
            leisure = self.omega - effort
            consumption = team_output(effort, total_effort_prev) / team_size
            return consumption ** self.theta * leisure ** (1 - self.theta)

        def negative_utility(effort):
            return -utility(effort)

        res = minimize(negative_utility, self.omega / 2, bounds=[(0, self.omega)])
        self.effort = res.x[0]
        self.utility = -res.fun

    def choose_team(self, teams):
        max_utility = self.utility
        best_team = self.team

        for team in [None] + list(teams.values()):
            if team is None:  # Own startup
                a_j = random.uniform(0, 0.5)
                b_j = random.uniform(0.75, 1.25)
                beta_j = random.uniform(1.5, 2)
                team_output = lambda effort, _: a_j * effort + b_j * effort ** beta_j
                team_size = 1
                total_effort_prev = 0
            else:
                team_output = team.output
                team_size = len(team.members)
                total_effort_prev = sum(agent.effort for agent in team.members)

            self.choose_effort(team_output, team_size, total_effort_prev)

            if self.utility > max_utility:
                max_utility = self.utility
                best_team = team

        if best_team is None:
            best_team = Team(self)
            teams[best_team] = best_team

        if self.team != best_team:
            if self.team:
                self.team.remove_member(self)
            self.team = best_team
            best_team.add_member(self)

In [10]:
def simulate(num_agents, num_periods):
    agents = [Agent(random.uniform(0, 1), 1, set()) for _ in range(num_agents)]
    
    for agent in agents:
        num_friends = random.randint(2, 6)
        agent.network = set(random.sample(agents, num_friends))

    teams = {Team(agent): Team(agent) for agent in agents}

    total_efforts = []
    total_output = []
    total_income = []
    total_utility = []
    job_changes = 0

    for period in range(num_periods):
        active_agents = random.sample(agents, int(num_agents * 0.04))

        for agent in active_agents:
            old_team = agent.team
            agent.choose_team(teams)
            if agent.team != old_team:
                job_changes += 1

        efforts = [agent.effort for agent in agents]
        output = [agent.team.output(agent.effort, sum(a.effort for a in agent.team.members)) for agent in agents]
        income = [out / len(agent.team.members) for out, agent in zip(output, agents)]
        utility = [agent.utility for agent in agents]

        total_efforts.append(np.mean(efforts))
        total_output.append(np.mean(output))
        total_income.append(np.mean(income))
        total_utility.append(np.mean(utility))

        for team in list(teams.keys()):
            if not team.members:
                del teams[team]

    return agents, teams, total_efforts, total_output, total_income, total_utility, job_changes

In [ ]:
def analyze_period(agents, teams, period):
    # Характеристики распределения отдельных показателей
    efforts = [agent.effort for agent in agents]
    output = [agent.team.output(agent.effort, sum(a.effort for a in agent.team.members)) for agent in agents]
    income = [out / len(agent.team.members) for out, agent in zip(output, agents)]
    utility = [agent.utility for agent in agents]
    team_sizes = [len(team.members) for team in teams.values()]
    team_ages = [period - agent.team.founder.birth_period for agent in agents]

    print(f"Period: {period}")
    print(f"Effort - Mean: {np.mean(efforts):.2f}, Median: {np.median(efforts):.2f}, Min: {np.min(efforts):.2f}, Max: {np.max(efforts):.2f}")
    print(f"Output - Mean: {np.mean(output):.2f}, Median: {np.median(output):.2f}, Min: {np.min(output):.2f}, Max: {np.max(output):.2f}")
    print(f"Income - Mean: {np.mean(income):.2f}, Median: {np.median(income):.2f}, Min: {np.min(income):.2f}, Max: {np.max(income):.2f}")
    print(f"Utility - Mean: {np.mean(utility):.2f}, Median: {np.median(utility):.2f}, Min: {np.min(utility):.2f}, Max: {np.max(utility):.2f}")
    print(f"Team Size - Mean: {np.mean(team_sizes):.2f}, Median: {np.median(team_sizes):.2f}, Min: {np.min(team_sizes):.2f}, Max: {np.max(team_sizes):.2f}")
    print(f"Team Age - Mean: {np.mean(team_ages):.2f}, Median: {np.median(team_ages):.2f}, Min: {np.min(team_ages):.2f}, Max: {np.max(team_ages):.2f}")

    # Гистограммы распределений
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 3, 1)
    plt.hist(efforts, bins=20)
    plt.title("Effort Distribution")
    plt.subplot(2, 3, 2)
    plt.hist(output, bins=20)
    plt.title("Output Distribution")
    plt.subplot(2, 3, 3)
    plt.hist(income, bins=20)
    plt.title("Income Distribution")
    plt.subplot(2, 3, 4)
    plt.hist(utility, bins=20)
    plt.title("Utility Distribution")
    plt.subplot(2, 3, 5)
    plt.hist(team_sizes, bins=20)
    plt.title("Team Size Distribution")
    plt.subplot(2, 3, 6)
    plt.hist(team_ages, bins=20)
    plt.title("Team Age Distribution")
    plt.tight_layout()
    plt.show()

    # Взаимосвязь показателей
    plt.figure(figsize=(8, 6))
    plt.scatter(team_ages, team_sizes)
    plt.xlabel("Team Age")
    plt.ylabel("Team Size")
    plt.title("Team Size vs Team Age")
    plt.show()

    growth_rates = [len(agent.team.members) / (period - agent.team.founder.birth_period) for agent in agents]
    plt.figure(figsize=(8, 6))
    plt.scatter(team_ages, growth_rates)
    plt.xlabel("Team Age")
    plt.ylabel("Growth Rate")
    plt.title("Growth Rate vs Team Age")
    plt.show()

        # Распределения для фирм
    team_output = [sum(agent.effort for agent in team.members) for team in teams.values()]
    team_sizes = [len(team.members) for team in teams.values()]
    team_ages = [period - team.founder.birth_period for team in teams.values()]
    team_productivity = [out / size for out, size in zip(team_output, team_sizes)]
    team_growth_rates = [len(team.members) / (period - team.founder.birth_period) for team in teams.values()]

    print(f"\nFirm Distributions:")
    print(f"Output - Mean: {np.mean(team_output):.2f}, Median: {np.median(team_output):.2f}, Min: {np.min(team_output):.2f}, Max: {np.max(team_output):.2f}")
    print(f"Size - Mean: {np.mean(team_sizes):.2f}, Median: {np.median(team_sizes):.2f}, Min: {np.min(team_sizes):.2f}, Max: {np.max(team_sizes):.2f}")
    print(f"Age - Mean: {np.mean(team_ages):.2f}, Median: {np.median(team_ages):.2f}, Min: {np.min(team_ages):.2f}, Max: {np.max(team_ages):.2f}")
    print(f"Productivity - Mean: {np.mean(team_productivity):.2f}, Median: {np.median(team_productivity):.2f}, Min: {np.min(team_productivity):.2f}, Max: {np.max(team_productivity):.2f}")
    print(f"Growth Rate - Mean: {np.mean(team_growth_rates):.2f}, Median: {np.median(team_growth_rates):.2f}, Min: {np.min(team_growth_rates):.2f}, Max: {np.max(team_growth_rates):.2f}")

    # Распределения для агентов
    agent_income = [agent.team.output(agent.effort, sum(a.effort for a in agent.team.members)) / len(agent.team.members) for agent in agents]
    agent_tenure = [period - agent.birth_period for agent in agents]
    agent_effort = [agent.effort for agent in agents]
    agent_utility = [agent.utility for agent in agents]
    agent_network_size = [len(agent.network) for agent in agents]

    print(f"\nAgent Distributions:")
    print(f"Income - Mean: {np.mean(agent_income):.2f}, Median: {np.median(agent_income):.2f}, Min: {np.min(agent_income):.2f}, Max: {np.max(agent_income):.2f}")
    print(f"Tenure - Mean: {np.mean(agent_tenure):.2f}, Median: {np.median(agent_tenure):.2f}, Min: {np.min(agent_tenure):.2f}, Max: {np.max(agent_tenure):.2f}")
    print(f"Effort - Mean: {np.mean(agent_effort):.2f}, Median: {np.median(agent_effort):.2f}, Min: {np.min(agent_effort):.2f}, Max: {np.max(agent_effort):.2f}")
    print(f"Utility - Mean: {np.mean(agent_utility):.2f}, Median: {np.median(agent_utility):.2f}, Min: {np.min(agent_utility):.2f}, Max: {np.max(agent_utility):.2f}")
    print(f"Network Size - Mean: {np.mean(agent_network_size):.2f}, Median: {np.median(agent_network_size):.2f}, Min: {np.min(agent_network_size):.2f}, Max: {np.max(agent_network_size):.2f}")

    # Гистограммы распределений для фирм
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 3, 1)
    plt.hist(team_output, bins=20)
    plt.title("Firm Output Distribution")
    plt.subplot(2, 3, 2)
    plt.hist(team_sizes, bins=20)
    plt.title("Firm Size Distribution")
    plt.subplot(2, 3, 3)
    plt.hist(team_ages, bins=20)
    plt.title("Firm Age Distribution")
    plt.subplot(2, 3, 4)
    plt.hist(team_productivity, bins=20)
    plt.title("Firm Productivity Distribution")
    plt.subplot(2, 3, 5)
    plt.hist(team_growth_rates, bins=20)
    plt.title("Firm Growth Rate Distribution")
    plt.tight_layout()
    plt.show()

    # Гистограммы распределений для агентов
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 3, 1)
    plt.hist(agent_income, bins=20)
    plt.title("Agent Income Distribution")
    plt.subplot(2, 3, 2)
    plt.hist(agent_tenure, bins=20)
    plt.title("Agent Tenure Distribution")
    plt.subplot(2, 3, 3)
    plt.hist(agent_effort, bins=20)
    plt.title("Agent Effort Distribution")
    plt.subplot(2, 3, 4)
    plt.hist(agent_utility, bins=20)
    plt.title("Agent Utility Distribution")
    plt.subplot(2, 3, 5)
    plt.hist(agent_network_size, bins=20)
    plt.title("Agent Network Size Distribution")
    plt.tight_layout()
    plt.show()

In [11]:
# Базовая конфигурация
num_agents = 120_000_000
num_periods = 1  # Один модельный период = один календарный месяц

agents, teams, efforts, output, income, utility, job_changes = simulate(num_agents, num_periods)

print(f"Number of agents: {len(agents)}")
print(f"Number of teams: {len(teams)}")
print(f"Average efforts: {np.mean(efforts):.2f}")
print(f"Average output: {np.mean(output):.2f}")
print(f"Average income: {np.mean(income):.2f}")
print(f"Average utility: {np.mean(utility):.2f}")
print(f"Number of job changes: {job_changes}")